# Section 3 - Euclid-Mullin Sequences in mk+1

In [39]:
import main
import math
import random
import csv

In [40]:
def list_to_poly(coeffs : list):
    '''Takes an integer list and returns a polynomial with those integers as coefficients
    e.g. [1,2,3] returns 1 + 2x + 3x**2'''
    def poly(x):
        sum = 0
        expo = 0
        for i in range(len(coeffs)):
            sum += coeffs[-i-1]*(x**expo)
            expo += 1
        return sum
    return poly

def phi_prime(p : int):
    '''Returns the polynomial of degree p-1 with all coefficients 1
    i.e. Phi_p(x) for p prime'''
    return list_to_poly([1]*p)

def phi(m,x):
    '''Recursive function to calculate Phi_m(x) for any m
    using the recursive division equation presented in section 3.1'''
    if m == 1:
        return x - 1
    factors = main.factor(m)
    if len(factors) == 1:
        return phi_prime(m)(x)
    else:
        val = phi_prime(m)(x)
        for d in range(2,m):
            if m % d == 0:
                val //= phi(d,x)
        return val

def fermat(a : int, d : int, N : int): # Book page 88
    '''Calculates a^d mod N'''
    prod = 1
    a2j = a
    while d>0:
        if d % 2 == 1:
            prod *= a2j
            prod %= N
        d //= 2
        a2j **= 2
        a2j %= N
    return prod

def fermat_test(N : int, iter = 10, baselim = 313):
    '''Performs a fermat compositeness test with multiple prime bases
     to try to rule out if N is prime. Output is whether or not N is
     (probably) prime.'''
    primes = main.eratosthanes(baselim)
    for p in primes:
        if N % p == 0 and N != p:
            return False
    for i in range(iter):
        if math.gcd(primes[i],N) == 1:
            if fermat(primes[i],N-1,N) != 1:
                return False
    else:
        return True

def modular_pow(base, exponent, modulus): # Function from https://www.geeksforgeeks.org/pollards-rho-algorithm-prime-factorization/
    '''Efficiently computes large powers base**exponent mod modulus'''
    result = 1  # initialize result 
    while (exponent > 0):
        if (exponent & 1): # if y is odd, multiply base with result 
            result = (result * base) % modulus
        exponent = exponent >> 1 # exponent = exponent/2 
        base = (base * base) % modulus # base = base * base 
    return result

def PollardRho(n): # Function from https://www.geeksforgeeks.org/pollards-rho-algorithm-prime-factorization/
    '''Implementation of the Pollard Rho algorithm to find factors'''
    if (n == 1): # no prime divisor for 1 
        return n
    if (n % 2 == 0): # even number means one of the divisors is 2 
        return 2
    x = (random.randint(0, 2) % (n - 2)) # we will pick from the range [2, N) 
    y = x
    # the constant in f(x).
    # Algorithm can be re-run with a different c
    # if it throws failure for a composite. 
    c = (random.randint(0, 1) % (n - 1))
    # Initialize candidate divisor (or result) 
    d = 1
    # until the prime factor isn't obtained.
    # If n is prime, return n 
    while (d == 1):
        # Tortoise Move: x(i+1) = f(x(i)) 
        x = (modular_pow(x, 2, n) + c + n)%n
        # Hare Move: y(i+1) = f(f(y(i))) 
        y = (modular_pow(y, 2, n) + c + n)%n
        y = (modular_pow(y, 2, n) + c + n)%n
        # check gcd of |x-y| and n 
        d = math.gcd(abs(x - y), n)
        # retry if the algorithm fails to find prime factor
        # with chosen x and c 
        if (d == n):
            return PollardRho(n)
    return d

def factor(n):
    '''Function that finds the prime factorisation of n.
    First brute force checks all primes less than 10000
    Then checks if remaining number is composite
    Then repeatedly uses PollardRho algorithm to find larger factors'''
    factors = []
    for p in main.eratosthanes(10000):
        while n % p == 0:
            n //= p
            factors.append(p)
    while n > 1:
        if not fermat_test(n):
            a = PollardRho(n)
            factors.append(a)
            n //= a
        else:
            factors.append(n)
            return factors
    return factors

## Section 3.2 - The Method

#### Example for 4k+1

In [41]:
seq4 = []

print('For the EM sequence in 4k+1 using Phi_4(4a):')

for i in range(5):
    print(seq4)
    a = math.prod(seq4)
    print(f'a = {a}')
    n = phi(4,4*a)
    print(f'N = {n}')
    facs = factor(n)
    print(f'N has factors {facs}')
    seq4.append(min(facs))
print(seq4)

For the EM sequence in 4k+1 using Phi_4(4a):
[]
a = 1
N = 17
N has factors [17]
[17]
a = 17
N = 4625
N has factors [5, 5, 5, 37]
[17, 5]
a = 85
N = 115601
N has factors [115601]
[17, 5, 115601]
a = 9826085
N = 1544831142835601
N has factors [1457497, 1059920633]
[17, 5, 115601, 1457497]
a = 14321489409245
N = 3281680942385867185463520401
N has factors [56921, 57653255255281305413881]
[17, 5, 115601, 1457497, 56921]


In [42]:
seq2 = []

print('For the EM sequence in 4k+1 using Phi_4(2a):')

for i in range(5):
    print(seq2)
    a = math.prod(seq2)
    print(f'a = {a}')
    n = phi(4,2*a)
    print(f'N = {n}')
    facs = factor(n)
    print(f'N has factors {facs}')
    seq2.append(min(facs))
print(seq2)

For the EM sequence in 4k+1 using Phi_4(2a):
[]
a = 1
N = 5
N has factors [5]
[5]
a = 5
N = 101
N has factors [101]
[5, 101]
a = 505
N = 1020101
N has factors [1020101]
[5, 101, 1020101]
a = 515151005
N = 1061522231810040101
N has factors [53, 1613, 12417062216309]
[5, 101, 1020101, 53]
a = 27303003265
N = 2981815949154402640901
N has factors [29, 137, 8143721, 92159345497]
[5, 101, 1020101, 53, 29]


#### Example for 5k+1

In [44]:
seq5 = []

print('For the EM sequence in 5k+1 using Phi_5(5a)')

for i in range(4):
    print(seq5)
    a = math.prod(seq5)
    print(f'a = {a}')
    n = phi(5,5*a)
    print(f'N = {n}')
    facs = factor(n)
    print(f'N has factors {facs}')
    seq5.append(max(facs))
print(seq5)

For the EM sequence in 5k+1 using Phi_5(5a)
[]
a = 1
N = 781
N has factors [11, 71]
[71]
a = 71
N = 15927165881
N has factors [11, 1447924171]
[71, 1447924171]
a = 102802616141
N = 69806631955964597980945010096726406220841630981
N has factors [11, 31, 41, 311, 12011, 364490341, 3667185416586300605613786041]
[71, 1447924171, 3667185416586300605613786041]
a = 376996254699194635758149876070627087781
N = 12624906200404940689191978070045377494879513343566667250272050819055623766466187699285599250055383402208906248865456217060620363487695490094039741799829056181
N has factors [31, 251, 311, 941, 12541, 202008151, 2188475408941814570040456230961014091230117129857427401718038042961655096837762611229601339388318619248220635485794241546725519124149161]
[71, 1447924171, 3667185416586300605613786041, 2188475408941814570040456230961014091230117129857427401718038042961655096837762611229601339388318619248220635485794241546725519124149161]


In [45]:
print(f'This last N had {len('12624906200404940689191978070045377494879513343566667250272050819055623766466187699285599250055383402208906248865456217060620363487695490094039741799829056181')} terms')
print(f'and largest prime divisor of length {len('2188475408941814570040456230961014091230117129857427401718038042961655096837762611229601339388318619248220635485794241546725519124149161')}.')

This last N had 158 terms
and largest prime divisor of length 136.


In [46]:
print(f'The next N we would need to factorise would be \n {phi(5,5*math.prod(seq5))}')
print(f'with {(len('289597523737836948333310175321110068291402270014451675897378558088880524898007179204225717767787513913090352970778596645549030767522058130112211235736444030169895041368741368318715349686946474093541864791417011076484807418821591250813130670299094315653702922771929769715435522821693179105047445334951924274600291441984810298032479687681727210609474857128375740705518723357793629201875436186665099102761462201197605060016101130915706130436221483433563205840669841388203682083729890494523552126979222041818352002555553365403953739683650101599111347113923014340418474434287105527174602106533590567951886805506619347340122640918970906431167598849298021918016772404015220844262832584559983748332817638981'))} digits.')

The next N we would need to factorise would be 
 289597523737836948333310175321110068291402270014451675897378558088880524898007179204225717767787513913090352970778596645549030767522058130112211235736444030169895041368741368318715349686946474093541864791417011076484807418821591250813130670299094315653702922771929769715435522821693179105047445334951924274600291441984810298032479687681727210609474857128375740705518723357793629201875436186665099102761462201197605060016101130915706130436221483433563205840669841388203682083729890494523552126979222041818352002555553365403953739683650101599111347113923014340418474434287105527174602106533590567951886805506619347340122640918970906431167598849298021918016772404015220844262832584559983748332817638981
with 699 digits.


In [47]:
n = 289597523737836948333310175321110068291402270014451675897378558088880524898007179204225717767787513913090352970778596645549030767522058130112211235736444030169895041368741368318715349686946474093541864791417011076484807418821591250813130670299094315653702922771929769715435522821693179105047445334951924274600291441984810298032479687681727210609474857128375740705518723357793629201875436186665099102761462201197605060016101130915706130436221483433563205840669841388203682083729890494523552126979222041818352002555553365403953739683650101599111347113923014340418474434287105527174602106533590567951886805506619347340122640918970906431167598849298021918016772404015220844262832584559983748332817638981
print('But it is easy to verify that is has smallest prime divisor')
for p in main.eratosthanes(10000):
    if n % p == 0:
        print(p)

But it is easy to verify that is has smallest prime divisor
1171


#### For discussion around what a can be multipled by instead of m.

The following code computes the value of Phi_m(x) (mod m) for all residues classes of m and for m up to 1000 and stores the data in the `cycvals.csv` file.

In [48]:
# # Can take a while to run so has been commented out to avoid accidental running. Data is already in the required file.

# with open('cycvals.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
    
#     for m in range(1, 1001):
#         mods = []
#         for i in range(m):
#             a = phi2(m, i) % m
#             mods.append(a)
        
#         # Write the mods list as a new row in the CSV file
#         writer.writerow(mods)